In [10]:
import random
import numpy as np

arrival = []
serve = []
departure = []

def sim_witout_priority(env, name, sellers):

    print(f"Customer {name} joins queue at  {env.now:.2f}")
    arrival.append((name , f'{env.now:.2f}' ))
    with sellers.request() as request:
        yield request
    
        salesperson = sellers.users.index(request) + 1
        
        print(f"Customer {name} starts being served by a seller {salesperson} at time {env.now:.2f}")

        

        serve.append((name, f'{env.now:.2f}',f"salesperson: {salesperson}"))
        support_time = max(np.random.normal(10, 4), 1) #using max to prevent negative delay
        yield env.timeout(support_time)
        
        print(f'Customer {name} finishes being served by a seller {salesperson} at time {env.now:.2f}, seller spend {support_time:.2f} for customer {name}')
        departure.append((name, f'{env.now:.2f}'))
def generate_customers_without_pri(env, sellers , max_customers):
    customer_num = 0
    while customer_num < max_customers:
        customer_num += 1
        env.process(sim_witout_priority(env, customer_num, sellers))
        enter_queue_time = random.expovariate(0.13)  #beccause customers enter shop in random time
        yield env.timeout(enter_queue_time)


def sim_with_priority(env, name, sellers, priority_customers):
 
    print(f"Customer {name} joins queue at time {env.now:.2f}")

    if name in priority_customers:
        with sellers.request() as request:
            yield request

            print(f"Priority customer {name} starts being served by a seller at time {env.now:.2f}")

            support_time = max(np.random.normal(10, 4), 1)
            yield env.timeout(support_time)

            print(f"Priority customer {name} finishes being served by a seller at time {env.now:.2f}, seller spend {support_time:.2f} for customer {name}")
    else:
        with sellers.request(priority=1) as request:
            yield request

            print(f"Customer {name} starts being served by a seller at time {env.now:.2f}")

            support_time = max(np.random.normal(10, 4), 1)
            yield env.timeout(support_time)

            print(f"Customer {name} finishes being served by a seller at time {env.now:.2f}, seller spend {support_time:.2f} for customer {name}")

def generate_customers_with_pri(env, sellers, priority_customers, max_customers):
    customer_num = 0
    while customer_num < max_customers:
        customer_num += 1
        env.process(sim_with_priority(env, customer_num, sellers, priority_customers))
        enter_queue_time = random.expovariate(0.13)  
        yield env.timeout(enter_queue_time)





In [11]:
import random
import simpy

MAX_CUSTOMERS = 50
NUM_SELLERS = 2  
NUM_PRIORITY_CUSTOMERS = 10
MAXIMUM_TIME = 200

running_code = True


def setup_without_pri():
    print("\nCandy Shop simulation with priority customers start \n")
    env = simpy.Environment()
    sellers = simpy.Resource(env, capacity= NUM_SELLERS)
    env.process(generate_customers_without_pri(env, sellers , MAX_CUSTOMERS))
    env.run(until= MAXIMUM_TIME) 
    print("\nCandy Shop simulation without priority customers ended \n")

    
def setup_with_pri():
    print("\nCandy Shop simulation with priority customers start \n")
    env = simpy.Environment()
    sellers = simpy.PriorityResource(env, capacity= NUM_SELLERS)
    priority_customers = random.sample(range(1, MAX_CUSTOMERS + 1), NUM_PRIORITY_CUSTOMERS)
    env.process(generate_customers_with_pri(env, sellers, priority_customers, MAX_CUSTOMERS))
    env.run(until= MAXIMUM_TIME)
    print("\nCandy Shop simulation with priority customers ended \n")


In [12]:
setup_without_pri()


Candy Shop simulation with priority customers start 

Customer 1 joins queue at  0.00
Customer 1 starts being served by a seller 1 at time 0.00
Customer 2 joins queue at  3.46
Customer 2 starts being served by a seller 2 at time 3.46
Customer 1 finishes being served by a seller 1 at time 12.14, seller spend 12.14 for customer 1
Customer 2 finishes being served by a seller 2 at time 12.65, seller spend 9.19 for customer 2
Customer 3 joins queue at  12.99
Customer 3 starts being served by a seller 1 at time 12.99
Customer 4 joins queue at  15.54
Customer 4 starts being served by a seller 2 at time 15.54
Customer 5 joins queue at  18.20
Customer 6 joins queue at  21.08
Customer 3 finishes being served by a seller 1 at time 21.56, seller spend 8.57 for customer 3
Customer 5 starts being served by a seller 2 at time 21.56
Customer 7 joins queue at  26.32
Customer 4 finishes being served by a seller 2 at time 27.56, seller spend 12.02 for customer 4
Customer 6 starts being served by a selle

In [13]:
setup_with_pri()


Candy Shop simulation with priority customers start 

Customer 1 joins queue at time 0.00
Customer 1 starts being served by a seller at time 0.00
Customer 1 finishes being served by a seller at time 8.14, seller spend 8.14 for customer 1
Customer 2 joins queue at time 22.69
Customer 2 starts being served by a seller at time 22.69
Customer 3 joins queue at time 23.03
Priority customer 3 starts being served by a seller at time 23.03
Customer 2 finishes being served by a seller at time 28.40, seller spend 5.71 for customer 2
Customer 4 joins queue at time 33.28
Priority customer 4 starts being served by a seller at time 33.28
Priority customer 3 finishes being served by a seller at time 35.30, seller spend 12.27 for customer 3
Customer 5 joins queue at time 38.28
Priority customer 5 starts being served by a seller at time 38.28
Customer 6 joins queue at time 39.95
Customer 7 joins queue at time 39.98
Priority customer 4 finishes being served by a seller at time 42.71, seller spend 9.43 f